In [1]:
from fairsearchcore.models import FairScoreDoc
import math
import sys
import warnings
warnings.filterwarnings("ignore")
import fairsearchcore as fsc
import sys
import pandas as pd

In [2]:
def build_ranking_model(is_protected_list):
    size = len(is_protected_list)
    unfair_ranking = []
    for i in range(size):
        unfair_ranking.append(FairScoreDoc((size-i), (size-i), is_protected_list[i]))
    return unfair_ranking

In [3]:
def is_ranking_fair(ranking, p, k, alpha=0.1):
    #k = 20 # number of topK elements returned (value should be between 10 and 400)
    #p = 0.1 # proportion of protected candidates in the topK elements (value should be between 0.02 and 0.98) 
    #alpha = 0.1 # significance level (value should be between 0.01 and 0.15)
    
    fair = fsc.Fair(k, p, alpha)
    return fair.is_fair(ranking)

In [4]:
def get_min_size_elements(df, queries, source, country, min_elements=10, tag_name='query'):
    min_size = float('inf')
    min_query = ""
    """
    print("{: >20} {: >10}".format("Query","Size"))
    print("-----------------------------------------")
    """

    for q in queries:
        size = len(df[(df[tag_name]==q) & (df['source'] == source) ])
        print(size)
        print(q)

        if (size < min_size) and (size >= min_elements):
            min_size = size
            print (min_size)

            min_query = q
            print (q)
            print("\n")
            print("{: >20} {: >10}".format(q,size))
    
    
    print("\n")
    print("{: >20} {: >10}".format(q,size))
    
    return min_size

In [5]:
def sex_discrimination(row, protected_sex='female'):
#    return row['gender'] == protected_sex 
#    return (row['gender'] == protected_sex or None or 'unknown')        
    return (row['gender'] != "male"  )    
def num_star_discrimination(row,protected='num_stars'):
    num_star = row['num_stars']
    return num_star<1           
def num_reviews_discrimination(row,protected='num_reviews'):
    num_reviews = row['num_reviews']
    return num_reviews<1           
def num_star_and_gender_discrimination(row,protected='num_stars'):
    num_star = row['num_stars']
    return (num_star==0 and row['gender'] == 'female') 


In [15]:
for q in sorted(list(f_queries_co)):
    print(q)
    df_u1_res = df_u2[(df_u2['speciality'] == q) & (df_u2['source'] == 'top doctors') & (df_u2['country'] == 'Colombia')].sort_values('#')[:15]
    df_u1_res.head()
    if len(df_u1_res) == 15: # if it has the minimum number of elements (minimum number of elements without unknowns for all queries)
        is_protected_list = []
        
        for i in range(len(df_u1_res)):
            condition = num_reviews_discrimination(df_u1_res.iloc[i],'num_star')
            print(i)
            print (condition)
            if condition:
                is_protected_list.append(True)
            else:
                is_protected_list.append(False)

angiologia y cirugia vascular
cardiologia adultos
cirugia general
cirugia oral y maxilofacial
0
False
1
True
2
True
3
False
4
False
5
False
6
False
7
True
8
True
9
False
10
False
11
False
12
False
13
False
14
False
cirugia plastica estetica y reparadora
dermatologia
0
True
1
True
2
False
3
True
4
False
5
True
6
True
7
True
8
True
9
True
10
True
11
True
12
True
13
True
14
False
endocrinologia
gastroenterologia especialidad
ginecologia y obstetricia
0
True
1
True
2
False
3
True
4
True
5
True
6
True
7
True
8
False
9
False
10
True
11
False
12
False
13
True
14
True
medicina estetica
neurocirugia
0
True
1
True
2
True
3
True
4
True
5
False
6
False
7
True
8
False
9
False
10
False
11
False
12
False
13
False
14
False
odontologia y estomatologia
oftalmologia
0
True
1
False
2
True
3
True
4
True
5
False
6
True
7
False
8
True
9
False
10
True
11
True
12
True
13
True
14
True
otorrinolaringologia
0
False
1
True
2
False
3
True
4
True
5
True
6
True
7
True
8
True
9
False
10
False
11
False
12
False
13
True

In [16]:
get_fairness_metrics(df_u2_star_filtered, f_queries_co, 'top doctors', 'Colombia', num_star_discrimination, {'df':df_u2, 'protected_value':'num_star'}, min_size, tag_name='speciality')


NameError: name 'get_fairness_metrics' is not defined

In [6]:
def get_fairness_metrics(df, queries, source, country, discrimination_function, positional_arguments, n, tag_name='query'):
    protected_lists = []
    p_values = [0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98]
    print("{: >15} {: >10}".format("Query","p = True"))
    print('          '+'-'*20)
#    if discrimination_function == old_age_discrimination or discrimination_function == young_age_discrimination:
#        median = 35
#        print("Median age= ",median)

    for q in sorted(list(queries)):
        df_u1_res = df[(df[tag_name] == q) & (df['source'] == source) & (df['country'] == country)].sort_values('#')[:n]
        df_u1_res.head()
        if len(df_u1_res) == n: # if it has the minimum number of elements (minimum number of elements without unknowns for all queries)
            is_protected_list = []
            for i in range(len(df_u1_res)):
                condition = discrimination_function(df_u1_res.iloc[i],positional_arguments['protected_value'])
                if condition:
                    is_protected_list.append(True)
                else:
                    is_protected_list.append(False)
            #print(is_protected_list)
            protected_lists.append(is_protected_list)
            ranking = build_ranking_model(is_protected_list)

            is_fair_list = list([bool(is_ranking_fair(ranking,p, n)) for p in p_values])
            if True in is_fair_list:
                p_index = len(is_fair_list) - 1 - is_fair_list[::-1].index(True)
                p_max = p_values[p_index]
            else:
                p_max = "-"
            #print(is_fair_list)
            if len(q) > 15: 
                q = q[:11]+'.'
            print("{: >15} {: >10}".format(q,p_max))

In [7]:
%store -r df_u2
%store -r f_queries_co
#%store -r f_queries_mx
#%store -r f_queries_es
%store -r intersection
%store -r f_queries_co
%store -r queries_es
%store -r queries_mx
%store -r df_u2es
%store -r df_u2mx

In [8]:
df_u2sp=df_u2es
queries_sp=queries_es

In [9]:
df_u2 = df_u2[df_u2['speciality'].isin(intersection)] 
df_u2sp = df_u2es[df_u2es['speciality'].isin(intersection)] 
df_u2mx = df_u2mx[df_u2mx['speciality'].isin(intersection)] 


In [21]:
queries_mx.head()

0                        acupuntura
3                       alergologia
23                       andrologia
26    angiologia y cirugia vascular
46                       audiologia
Name: speciality, dtype: object

In [22]:
df_u2mx.head()

,source,country,speciality,#,gender,ethnicity,Is_foreigner,age,experience_level,was_awarded,has_photo,num_stars,num_reviews
162,top doctors,Mexico,cirugia oral y maxilofacial,0,male,hispanic,False,37.0,senior2-level,False,True,5.0,1
163,top doctors,Mexico,cirugia oral y maxilofacial,1,male,hispanic,False,35.0,senior2-level,False,True,5.0,3
164,top doctors,Mexico,cirugia oral y maxilofacial,1,male,hispanic,False,NaN,senior2-level,False,False,5.0,5
165,top doctors,Mexico,cirugia oral y maxilofacial,2,male,hispanic,False,53.0,senior2-level,False,True,0.0,0
166,top doctors,Mexico,cirugia oral y maxilofacial,2,female,hispanic,False,53.0,senior2-level,False,True,5.0,1


In [10]:
#colombia
#df_u2_sex_filtered=df_u2
df_u2_sex_filtered = df_u2[(df_u2['gender']!=None) & (df_u2['gender']!='unknown')]
df_u2_star_filtered = df_u2[(df_u2['num_stars']!=None) & (df_u2['num_stars']!='unknown')]
df_u2_reviews_filtered = df_u2[(df_u2['num_reviews']!=None) & (df_u2['num_reviews']!='unknown')]


In [11]:
#spain
#df_u2_sex_filtered=df_u2
df_u2_sex_filtered_sp = df_u2sp[(df_u2sp['gender']!=None) & (df_u2sp['gender']!='unknown')]
df_u2_star_filtered_sp = df_u2sp[(df_u2sp['num_stars']!=None) & (df_u2sp['num_stars']!='unknown')]
df_u2_reviews_filtered_sp = df_u2sp[(df_u2sp['num_reviews']!=None) & (df_u2sp['num_reviews']!='unknown')]


In [25]:
df_u2_sex_filtered_sp.head()

,source,country,speciality,#,gender,ethnicity,Is_foreigner,age,experience_level,was_awarded,has_photo,num_stars,num_reviews
2,top doctors,Spain,urologia,0,male,hispanic,False,NaN,senior2-level,False,True,5.0,12
3,top doctors,Spain,urologia,1,male,hispanic,False,NaN,senior2-level,False,True,5.0,5
4,top doctors,Spain,urologia,2,male,hispanic,False,34.0,senior2-level,False,True,5.0,3
5,top doctors,Spain,urologia,3,male,hispanic,False,45.0,NaN,False,True,5.0,1
6,top doctors,Spain,urologia,4,male,hispanic-italian,False,53.0,senior2-level,False,True,5.0,18


In [12]:
#Maxican
#df_u2_sex_filtered=df_u2
df_u2_sex_filtered_max = df_u2mx[(df_u2mx['gender']!=None) & (df_u2mx['gender']!='unknown')]
df_u2_star_filtered_max = df_u2mx[(df_u2mx['num_stars']!=None) & (df_u2mx['num_stars']!='unknown')]
df_u2_reviews_filtered_max= df_u2mx[(df_u2mx['num_reviews']!=None) & (df_u2mx['num_reviews']!='unknown')]


In [27]:
#colombia_min
min_size_co = get_min_size_elements(df_u2_star_filtered, f_queries_co, source='top doctors', country='Colombia', tag_name='speciality')


0
angiologia y cirugia vascular
0
cardiologia adultos
0
cirugia general
20
cirugia oral y maxilofacial
20
cirugia oral y maxilofacial


cirugia oral y maxilofacial         20
0
cirugia plastica estetica y reparadora
20
dermatologia
0
endocrinologia
0
gastroenterologia especialidad
20
ginecologia y obstetricia
0
medicina estetica
20
neurocirugia
0
odontologia y estomatologia
20
oftalmologia
20
otorrinolaringologia
20
pediatria
20
psicologia
20
psiquiatria
0
reumatologia
0
traumatologia y ortopedia
20
urologia


            urologia         20


In [28]:
df_u2_sex_filtered_sp.head()

,source,country,speciality,#,gender,ethnicity,Is_foreigner,age,experience_level,was_awarded,has_photo,num_stars,num_reviews
2,top doctors,Spain,urologia,0,male,hispanic,False,NaN,senior2-level,False,True,5.0,12
3,top doctors,Spain,urologia,1,male,hispanic,False,NaN,senior2-level,False,True,5.0,5
4,top doctors,Spain,urologia,2,male,hispanic,False,34.0,senior2-level,False,True,5.0,3
5,top doctors,Spain,urologia,3,male,hispanic,False,45.0,NaN,False,True,5.0,1
6,top doctors,Spain,urologia,4,male,hispanic-italian,False,53.0,senior2-level,False,True,5.0,18


In [29]:
queries_sp.head()

0          urologia infantil
2                   urologia
22          unidad del dolor
39    traumatologia infantil
48             traumatologia
Name: speciality, dtype: object

In [13]:
#spain_min
min_size_sp = get_min_size_elements(df_u2_sex_filtered_sp, queries_sp, source='top doctors', country='Spain', tag_name='speciality')

0
urologia infantil
20
urologia
20
urologia


            urologia         20
0
unidad del dolor
0
traumatologia infantil
0
traumatologia
0
sexologia
0
reumatologia
0
reproduccion asistida
0
radiologia
0
psiquiatria infantil y adolescente especialidad
20
psiquiatria
20
psicologia
0
podologia especialidad
20
pediatria
0
ozonoterapia especialidad
20
otorrinolaringologia
0
osteopatia
0
oncologia radioterapica
0
oncologia medica especialidad
0
oftalmologia infantil especialidad
20
oftalmologia
0
odontologia y estomatologia
0
nutricion y dietetica
0
neurorradiologia intervencionista
0
neurorehabilitacion especialidad
0
neurologia infantil especialidad
0
neurologia
0
neurofisiologia clinica especialidad
20
neurocirugia
0
neumologia infantil
0
neumologia
0
nefrologia infantil
0
nefrologia especialidad
0
medicina regenerativa especialidad
0
medicina nuclear especialidad
0
medicina interna
0
medicina intensiva
0
medicina forense
0
medicina fisica y rehabilitacion
0
medicina familiar
0
medicina 

In [14]:
#Maxico_min
min_size_mx = get_min_size_elements(df_u2_sex_filtered_max, queries_mx, source='top doctors', country='Mexico', tag_name='speciality')

0
acupuntura
0
alergologia
0
andrologia
0
angiologia y cirugia vascular
0
audiologia
0
cardiologia especialidad
0
cardiologia pediatrica
0
cirugia bariatrica especialidad
0
cirugia cardiaca
0
cirugia cardiaca pediatrica
0
cirugia de columna especialidad
0
cirugia general y bariatrica
0
cirugia oncologica especialidad
20
cirugia oral y maxilofacial
20
cirugia oral y maxilofacial


cirugia oral y maxilofacial         20
0
cirugia pediatrica
0
cirugia plastica estetica y reconstructiva
0
cirugia toracica
0
coloproctologia
20
dermatologia
0
endocrinologia
0
endoscopia digestiva
0
fisiatria y rehabilitacion
0
fisioterapia
0
foniatria
0
gastroenterologia especialidad
0
genetica
0
geriatria
20
ginecologia y obstetricia
0
hematologia
0
inmunologia
0
mastologia
0
medicina del deporte especialidad
0
medicina del dolor
0
medicina familiar y comunitaria
0
medicina interna
0
medicina materno fetal
0
medicina nuclear especialidad
0
medicina y terapia intensiva
0
microbiologia e infectologia
0
nefrol

In [32]:
#colombia_fairness
get_fairness_metrics(df_u2_sex_filtered, f_queries_co, 'top doctors', 'Colombia', sex_discrimination, {'df':df_u2, 'protected_value':'female'}, min_size_co, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.3
   dermatologia        0.9
   ginecologia.        0.4
   neurocirugia        0.1
   oftalmologia        0.5
   otorrinolar.        0.4
      pediatria        0.6
     psicologia        0.8
    psiquiatria        0.6
       urologia        0.1


In [63]:
#spain_fairness
get_fairness_metrics(df_u2_sex_filtered_sp, queries_sp, 'top doctors', 'Spain', sex_discrimination, {'df':df_u2sp, 'protected_value':'female'}, min_size_sp, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.1
   dermatologia        0.6
   ginecologia.        0.4
   neurocirugia        0.2
   oftalmologia        0.3
   otorrinolar.        0.2
      pediatria        0.5
     psicologia        0.8
    psiquiatria        0.5
       urologia        0.2


In [58]:
#Mexico_fairness
get_fairness_metrics(df_u2_sex_filtered_max, queries_mx, 'top doctors', 'Mexico', sex_discrimination, {'df':df_u2mx, 'protected_value':'female'}, min_size_mx, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.3
   dermatologia        0.8
   ginecologia.        0.4
   neurocirugia        0.1
   oftalmologia        0.7
   otorrinolar.        0.7
      pediatria        0.5
     psicologia        0.8
    psiquiatria        0.4
       urologia        0.1


In [16]:
#colombia_fair_number_of_star
get_fairness_metrics(df_u2_star_filtered, f_queries_co, 'top doctors', 'Colombia', num_star_discrimination, {'df':df_u2, 'protected_value':'num_star'}, min_size, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.8
   dermatologia        0.3
   ginecologia.        0.4
   neurocirugia        0.4
   oftalmologia        0.4
   otorrinolar.        0.5
      pediatria        0.7
     psicologia        0.6
    psiquiatria        0.6
       urologia        0.1


In [17]:
#spain_fair_number_of_star
get_fairness_metrics(df_u2_star_filtered_sp, queries_sp, 'top doctors', 'Spain', num_star_discrimination, {'df':df_u2sp, 'protected_value':'num_star'}, min_size_sp, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.5
   dermatologia        0.1
   ginecologia.        0.2
   neurocirugia        0.2
   oftalmologia        0.3
   otorrinolar.        0.3
      pediatria        0.2
     psicologia        0.6
    psiquiatria        0.4
       urologia        0.2


In [18]:
#Mexico_fair_number_of_star
get_fairness_metrics(df_u2_star_filtered_max, queries_mx, 'top doctors', 'Mexico', num_star_discrimination, {'df':df_u2mx, 'protected_value':'num_star'}, min_size_sp, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.6
   dermatologia        0.2
   ginecologia.        0.3
   neurocirugia        0.4
   oftalmologia        0.3
   otorrinolar.        0.1
      pediatria        0.7
     psicologia        0.5
    psiquiatria        0.4
       urologia        0.3


In [19]:
#colombia_fair_number_of_reviews
get_fairness_metrics(df_u2_reviews_filtered, f_queries_co, 'top doctors', 'Colombia', num_reviews_discrimination, {'df':df_u2, 'protected_value':'num_reviews'}, min_size, tag_name='speciality')

          Query   p = True
          --------------------
   cirugia ora.        0.8
   dermatologia        0.3
   ginecologia.        0.4
   neurocirugia        0.4
   oftalmologia        0.4
   otorrinolar.        0.5
      pediatria        0.7
     psicologia        0.6
    psiquiatria        0.6
       urologia        0.1


In [20]:
#Mexico_fair_number_of_reviews
get_fairness_metrics(df_u2_reviews_filtered_max,queries_mx, 'top doctors', 'Mexico', num_reviews_discrimination, {'df':df_u2mx, 'protected_value':'num_reviews'}, min_size, tag_name='speciality')

          Query   p = True
          --------------------
   cirugia ora.        0.6
   dermatologia        0.2
   ginecologia.        0.3
   neurocirugia        0.4
   oftalmologia        0.3
   otorrinolar.        0.1
      pediatria        0.7
     psicologia        0.5
    psiquiatria        0.4
       urologia        0.3


In [39]:
#Spain_fair_number_of_reviews
get_fairness_metrics(df_u2_reviews_filtered_sp,queries_sp, 'top doctors', 'Spain', num_reviews_discrimination, {'df':df_u2sp, 'protected_value':'num_reviews'}, min_size, tag_name='speciality')

          Query   p = True
          --------------------
   cirugia ora.        0.7
   dermatologia        0.9
   ginecologia.        0.9
   neurocirugia        0.9
   oftalmologia        0.8
   otorrinolar.        0.9
      pediatria        0.9
     psicologia        0.7
    psiquiatria        0.8
       urologia        0.8


In [119]:
#Mexico_fair_number_of_star_and famel
get_fairness_metrics(df_u2_star_filtered_max, queries_mx, 'top doctors', 'Mexico', num_star_and_gender_discrimination, {'df':df_u2mx, 'protected_value':'num_star'}, min_size_sp, tag_name='speciality')

          Query   p = True
          --------------------
   cirugia ora.        0.5
   dermatologia        0.1
   ginecologia.        0.3
   neurocirugia        0.4
   oftalmologia        0.3
   otorrinolar.        0.1
      pediatria        0.4
     psicologia        0.4
    psiquiatria        0.3
       urologia        0.3


In [132]:
#colombia_fair_number_of_star_and_gender
get_fairness_metrics(df_u2_star_filtered, f_queries_co, 'top doctors', 'Colombia', num_star_and_gender_discrimination, {'df':df_u2, 'protected_value':'num_star'}, min_size, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.3
   dermatologia        0.2
   ginecologia.        0.2
   neurocirugia        0.1
   oftalmologia        0.3
   otorrinolar.        0.4
      pediatria        0.4
     psicologia        0.5
    psiquiatria        0.4
       urologia        0.1


In [133]:
#spain_fair_number_of_star_and_gender
get_fairness_metrics(df_u2_star_filtered_sp, queries_sp, 'top doctors', 'Spain', num_star_and_gender_discrimination, {'df':df_u2sp, 'protected_value':'num_star'}, min_size_sp, tag_name='speciality')


          Query   p = True
          --------------------
   cirugia ora.        0.1
   dermatologia        0.1
   ginecologia.        0.1
   neurocirugia        0.1
   oftalmologia        0.2
   otorrinolar.        0.1
      pediatria        0.2
     psicologia        0.5
    psiquiatria        0.2
       urologia        0.1


In [107]:
c=0
for q in sorted(f_queries_co):
    if c==0:
        print(c)
        dfObj =  df_u2_sex_filtered[(df_u2_sex_filtered['speciality'] == q) & (df_u2_sex_filtered['source'] == 'top doctors')].sort_values('#')[:20]
    else:
        df_20 =  df_u2_sex_filtered[(df_u2_sex_filtered['speciality'] == q) & (df_u2_sex_filtered['source'] == 'top doctors')].sort_values('#')[:20]
        dfObj=dfObj+df_20
    c+=1
    


0


In [27]:
export_csv = df_u2_sex_filtered.to_csv (r'df_u2_sex_filtered_unknown_famel.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [111]:
def get_metrics_gender(df, source, country, queries, tag_queries='speciality'):
    print("Country: " + country + " Source: " + source)
    print("{: >20} {: >10} {: >10} {: >10} {: >10}{: >10} {: >10}".format("Q","%M#10", "%F#10", "%U#10","%M#20", "%F#20", "%U#20"))
#    s="Country: " + country + " Source: " + source+'\n'
 #   s=s+"{: >20} {: >10} {: >10} {: >10} {: >10}{: >10} {: >10}".format("Q","%M#10", "%F#10", "%U#10","%M#20", "%F#20", "%U#20")+'\n'
    for q in sorted(queries):
        df_10 = df[(df[tag_queries] == q) & (df['source'] == source)].sort_values('#')[:10]
        df_20 = df[(df[tag_queries] == q) & (df['source'] == source)].sort_values('#')[:20]
        
        m_10 = round((len(df_10[(df_10['gender'] == 'male')]) / float(10)) * 100,3)
        f_10 = round((len(df_10[(df_10['gender'] == 'female')]) / float(10)) * 100,3)
        u_10 = round((len(df_10[(df_10['gender'] == 'unknown')]) / float(10)) * 100,3)
        
        m_20 = round((len(df_20[(df_20['gender'] == 'male')]) / float(20)) * 100,3)
        f_20 = round((len(df_20[(df_20['gender'] == 'female')]) / float(20)) * 100,3)
        u_20 = round((len(df_20[(df_20['gender'] == 'unknown')]) / float(20)) * 100,3)
       # s=s+"{: >20} {: >10} {: >10} {: >10} {: >10}{: >10} {: >10}".format(q,m_10,f_10,u_10,m_20,f_20,u_20)+'\n'
        # If the query has assigned less than 10 people, the sum of proportions will not result in 100%
        print("{: >20} {: >10} {: >10} {: >10} {: >10}{: >10} {: >10}".format(q,m_10,f_10,u_10,m_20,f_20,u_20))
        #print (s)